Na (malinko změněném) [převzatém příkladu](http://nbviewer.ipython.org/github/cs109/content/blob/master/lec_04_wrangling.ipynb) si ukážeme, jako pomocí [Pandas](http://pandas.pydata.org/) spolu s [Matplotlib](http://matplotlib.org/) a [ggplot](https://github.com/yhat/ggplot/) analyzovat data. Pandas je z velké části inspirován R-kem, konkrétně `data.frame` třídou, která se v Pandas nazývá `DataFrame`. Jedná se o 2D tabulární strukturu, podobnou relační databázi (SQL) nebo tabulce Excelu. Pandas využívá NumPy pro většinu výpočtů, které jsou tím pádem velice rychlé, a zároveň umožňuje velice flexibilní manipulaci s daty.
<!-- TEASER_END -->

# Pandas

Materiály převzaty z: Companion to Lecture 4 of [Harvard CS109: Data Science](http://cs109.org) course.

In [ ]:
# pokud nemáme knihovnu pandas, tak ji nainstalujeme
# !pip install pandas -U

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Máme připravený dataset s informacemi o top filmech dle [IMDB](http://www.imdb.com/). Soubor imdb_top_10000.txt)

In [ ]:
# prvních několik řádků
!head imdb_top_10000.txt

## 1. Sestavte DataFrame

Textový soubor je oddělen tabulátory a nemá žádná záhlaví sloupců. Nastavíme příslušná klíčová slova v `pd.read_csv`, aby se dataset bez problému načetl.

In [ ]:
names = ['imdbID', 'title', 'year', 'score', 'votes', 'runtime', 'genres']
data = pd.read_csv('imdb_top_10000.txt', delimiter='\t', names=names).dropna()
print("Number of rows: %i" % data.shape[0])
data.head()  # print the first 5 rows

Všimněte si pěkného html formátování výsledného DataFrame.

## 2. Vyčistěte DataFrame

V tomto okamžiku existuje několik problémů s DataFrame:

1. Sloupec runtime popisuje číslo, ale je uložen jako řetězec
2. Sloupec žánry není atomický – sdružuje několik žánrů dohromady. To ztěžuje například extrahování filmů, které jsou komedie.
3. Rok filmu se opakuje ve sloupci název a rok

### Oprava sloupce runtime

Následující úryvek převede řetězec jako '142 minut.' na číslo 142:

In [ ]:
dirty = '142 mins.'
number, text = dirty.split(' ')
clean = int(number)
print(number)

Můžeme to udělat najednou pro všechny záznamy pomocí list comprehension

In [ ]:
clean_runtime = [float(r.split(' ')[0]) for r in data.runtime]
data['runtime'] = clean_runtime
data.head()

### Rozdělení žánrů
Můžeme použít koncept *indicator variables* k rozdělení sloupce žánry do mnoha sloupců. Každý nový sloupec bude odpovídat jednomu žánru a každá buňka bude mít hodnotu True nebo False.

In [ ]:
# vyrobíme seznam unikátních žánrů

genres = set()
for m in data.genres:
    genres.update(g for g in m.split('|'))
genres = sorted(genres)
print(genres)

In [ ]:
# každý žánr přidáme jako nový sloupec do tabulky
for genre in genres:
    data[genre] = [genre in movie.split('|') for movie in data.genres]
         
data.head()

### Odebírání roku z názvu
Každý prvek můžeme opravit odstraněním posledních 7 znaků

In [ ]:
data['title'] = [t[0:-7] for t in data.title]
data.head()

## 3. Prozkoumejte globální vlastnosti

Dále se seznámíme s některými základními, globálními souhrny tabulky DataFrame.

### V příslušných sloupcích zavolejte `describe`.

In [ ]:
data[['score', 'runtime', 'year', 'votes']].describe(include='all')

In [ ]:
# hmmm, a runtime = 0?!
print(len(data[data.runtime == 0]))


In [ ]:
# lepší by bylo nahradit nulou za NaN, 0 smysl nedává, ať nás neplete
data.runtime[data.runtime==0] = np.nan

In [ ]:
data.runtime.describe()

### Make some basic plots

In [ ]:
plt.hist(data.year, bins=np.arange(1950, 2013), color='#cccccc')
plt.xlabel("Release Year")

**Grafy pomocí [ggplot](https://github.com/yhat/ggplot/) balíčku. Pro uživatele R.**

In [ ]:
# instalace pomocí knihovny plotnine
# !pip install plotnine -U

In [ ]:
import plotnine as p9

In [ ]:
p = p9.ggplot(data, p9.aes(x='year'))
p + p9.geom_histogram(binwidth=1) + p9.ggtitle("Movies per year histogram")


In [ ]:
plt.hist(data.score, bins=20, color='#cccccc',density=True)
plt.xlabel("IMDB rating")

In [ ]:
plt.hist(data.runtime.dropna(), bins=50, color='#cccccc',density=True)
plt.xlabel("Runtime distribution")

In [ ]:
#hmm, more bad, recent movies. Real, or a selection bias?

plt.scatter(data.year, data.score, lw=0, alpha=.08, color='k')
plt.xlabel("Year")
plt.ylabel("IMDB Rating")

In [ ]:
plt.scatter(data.votes, data.score, lw=0, alpha=.2, color='k')
plt.xlabel("Number of Votes")
plt.ylabel("IMDB Rating")
plt.xscale('log')

### Vybírání prvků dle podmínek

In [ ]:
# low-score movies with lots of votes
data[(data.votes > 9e4) & (data.score < 5)][['title', 'year', 'score', 'votes', 'genres']]

In [ ]:
# The lowest rated movies
data[data.score == data.score.min()][['title', 'year', 'score', 'votes', 'genres']]

In [ ]:
# The highest rated movies
data[data.score == data.score.max()][['title', 'year', 'score', 'votes', 'genres']]

### Spusťte agregační funkce jako `sum` na několika řádcích nebo sloupcích

*Jaké žánry jsou nejčastější?*

In [ ]:
#sum sums over rows by default
genre_count = np.sort(data[genres].sum())[::-1]
pd.DataFrame({'Genre': genres, 'Genre Count': genre_count})

*Kolik žánrů má film v průměru?*

In [ ]:
#axis=1 sums over columns instead
genre_count = data[genres].sum(axis=1) 
print(f"Average movie has {genre_count.mean()} genres")
genre_count.describe()

## Prozkoumejte vlastnosti skupin

Rozdělme filmy podle desetiletí

In [ ]:
decade =  (data.year // 10) * 10

tyd = data[['title', 'year']]
tyd['decade'] = decade

tyd.head()

[GroupBy](http://pandas.pydata.org/pandas-docs/dev/groupby.html) will gather movies into groups with equal decade values

In [ ]:
# mean score for all movies in each decade
decade_mean = data.groupby(decade).score.mean()
decade_mean.name = 'Decade Mean'
print(decade_mean)

In [ ]:
# vykreslíme si graf (předchozí scatter plot) s průměrem hodnocení za dekádu
plt.plot(decade_mean.index, decade_mean.values, 'o-',
        color='r', lw=3, label='Decade Average')
plt.scatter(data.year, data.score, alpha=.04, lw=0, color='k')
plt.xlabel("Year")
plt.ylabel("Score")
plt.legend(frameon=False)

Můžeme jít ještě dále a vypočítat rozptyl v každédekádě

In [ ]:
grouped_scores = data.groupby(decade).score

mean = grouped_scores.mean()
std = grouped_scores.std()
print(std)

In [ ]:
# a přikreslit tento udaj jako "fill" kolem průměru
plt.plot(decade_mean.index, decade_mean.values, 'o-',
        color='r', lw=3, label='Decade Average')
plt.fill_between(decade_mean.index, (decade_mean + std).values,
                 (decade_mean - std).values, color='r', alpha=.2)
plt.scatter(data.year, data.score, alpha=.04, lw=0, color='k')
plt.xlabel("Year")
plt.ylabel("Score")
plt.legend(frameon=False)

Objekt GroupBy můžete také iterovat. Každá iterace poskytuje dvě proměnné: jednu z odlišných hodnot skupinového klíče a podmnožinu datového rámce, kde se klíč rovná této hodnotě. Jak najít nejoblíbenější film každý rok:

In [ ]:
for year, subset in data.groupby('year'):
    print(year, subset[subset.score == subset.score.max()].title.values)

### Malé násobky

Rozdělme filmy podle žánru a podívejme se, jak se liší jejich rok vydání/doba spuštění/skóre IMDB.
Distribuce pro všechny filmy je zobrazena jako šedé pozadí.

Toto není standardní groupby, takže zde nemůžeme použít metodu `groupby`. Je potřeba ruční smyčka

In [ ]:
#create a 4x6 grid of plots.
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(12, 8), 
                         tight_layout=True)

bins = np.arange(1950, 2013, 3)
for ax, genre in zip(axes.ravel(), genres):
    ax.hist(data[data[genre] == 1].year, density=True,
            bins=bins, histtype='stepfilled', color='r', alpha=.3, ec='none')
    ax.hist(data.year, bins=bins, histtype='stepfilled', ec='None', density=True, zorder=0, color='#cccccc')
    
    ax.annotate(genre, xy=(1955, 3e-2), fontsize=14)
    ax.xaxis.set_ticks(np.arange(1950, 2013, 30))
    ax.set_yticks([])
    ax.set_xlabel('Year')

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(12, 8), tight_layout=True)

bins = np.arange(30, 240, 10)

for ax, genre in zip(axes.ravel(), genres):
    ax.hist(data[data[genre] == 1].runtime, density=True,
            bins=bins, histtype='stepfilled', color='r', ec='none', alpha=.3)
               
    ax.hist(data.runtime, bins=bins, density=True,
            histtype='stepfilled', ec='none', color='#cccccc',
            zorder=0)
    
    ax.set_xticks(np.arange(30, 240, 60))
    ax.set_yticks([])
    ax.set_xlabel("Runtime [min]")
    ax.annotate(genre, xy=(230, .02), ha='right', fontsize=12)

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(12, 8), tight_layout=True)

bins = np.arange(0, 10, .5)

for ax, genre in zip(axes.ravel(), genres):
    ax.hist(data[data[genre] == 1].score, density=True,
            bins=bins, histtype='stepfilled', color='r', ec='none', alpha=.3)
               
    ax.hist(data.score, bins=bins,
            histtype='stepfilled', ec='none', color='#cccccc', density=True,
            zorder=0)
    
    ax.set_yticks([])
    ax.set_xlabel("Score")
    ax.annotate(genre, xy=(0, .2), ha='left', fontsize=12)

## Other Resources

* Chapters 5 and 7 of [Python for Data Analysis](http://www.amazon.com/Python-Data-Analysis-Wes-McKinney/dp/1449319793)
* Jupyter Notebook on [Data wrangling with Pandas](http://nbviewer.ipython.org/urls/gist.github.com/fonnesbeck/5850413/raw/3a9406c73365480bc58d5e75bc80f7962243ba17/2.+Data+Wrangling+with+Pandas.ipynb)
* [Lecture 4](https://drive.google.com/folderview?id=0BxYkKyLxfsNVd0xicUVDS1dIS0k&usp=sharing) of Harvard's CS 109: Data Science